#### Total for 1km 

In [1]:
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json

# Load population data and business data
population_data = pd.read_csv('../../Cleaned Data/Cleaned_population_data.csv')  
business_data = pd.read_csv('../Foursquare/final_coffeeshops_dataset_cleaned.csv')  

# Convert the 'geometry_region' (which is a string of coordinates) into a Polygon
def convert_to_polygon(geometry_string):
    coordinates = json.loads(geometry_string)  # Convert the string into a list of coordinates
    return Polygon(coordinates[0])  # Create a Polygon from the coordinates

population_data['geometry_region'] = population_data['geometry_region'].apply(convert_to_polygon)

# Function to calculate population within a 1km radius around the business
def match_population_to_business(business_row):
    business_point = Point(business_row['Longitude'], business_row['Latitude'])
    total_population = 0

    # Create a 1km buffer around the business point (~0.009 degrees)
    buffer = business_point.buffer(0.009)  # 1 km buffer (approximate in degrees)

    for _, pop_row in population_data.iterrows():
        if pop_row['geometry_region'].intersects(buffer):  # Check if the population polygon intersects the buffer
            total_population += pop_row['population_count']
    
    return total_population

# Add the matched population within 1km to the business data
business_data['Population Within 1km'] = business_data.apply(lambda row: match_population_to_business(row), axis=1)

# Overwrite the same file with updated data
business_data.to_csv('../Foursquare/final_coffeeshops_dataset_cleaned.csv', index=False)

print("Coffeeshops data updated with population within 1km.")


Coffeeshops data updated with population within 1km.


#### Visualization

In [16]:
import pandas as pd
import folium
import json
from shapely.geometry import Point, Polygon
import geopandas as gpd

# Load business and population data
business_data = pd.read_csv("../../Collected Data/Old data/restaurants_dataset_cleaned2.csv")  
population_data = pd.read_csv("../../Cleaned Data/Cleaned_population_data.csv")  

# Create Folium map centered around business locations
map_center = [business_data['POI Latitude'].mean(), business_data['POI Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Function to convert geometry_region to a valid Polygon
def convert_to_polygon(geometry_string):
    try:
        coordinates = json.loads(geometry_string)  # Convert string to list
        if isinstance(coordinates, list) and len(coordinates) > 0:
            return Polygon([tuple(coord) for coord in coordinates[0]])  # Ensure tuple format
    except Exception as e:
        print(f"Error converting polygon: {e}")
    return None  # Return None if conversion fails

# Convert population polygons 
population_data['geometry_region'] = population_data['geometry_region'].apply(convert_to_polygon)
population_data = population_data.dropna(subset=['geometry_region'])  # Remove invalid polygons

# Plot population squares FIRST 
for _, row in population_data.iterrows():
    polygon = row['geometry_region']
    population_count = row['population_count']
    
    folium.GeoJson(
        {
            "type": "Feature",
            "geometry": {"type": "Polygon", "coordinates": [list(polygon.exterior.coords)]},
            "properties": {"population": population_count}
        },
        name="Population Squares",
        style_function=lambda feature: {
            "fillColor": "#ff7800",
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.5
        },
        popup=folium.Popup(f"Population: {population_count}", max_width=300),
        pane="overlayPane"  # Moves population squares to foreground 
    ).add_to(m)

# Plot businesses and their 1km buffer
for _, row in business_data.iterrows():
    business_point = [row['POI Latitude'], row['POI Longitude']]
    business_location = Point(row['POI Longitude'], row['POI Latitude'])  # Convert to Shapely Point
    
    # Create a 1km buffer
    buffer_radius = 0.009  # Approx 1km in degrees (for lat/lon)
    business_buffer = business_location.buffer(buffer_radius)
    
    # Calculate total population within 1km buffer
    total_population = sum(
        pop_row['population_count'] for _, pop_row in population_data.iterrows() 
        if pop_row['geometry_region'].intersects(business_buffer)
    )

    # Marker for the business
    folium.Marker(
        location=business_point,
        popup=row['POI Name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)
    
    # 1km radius circle (SEND THIS TO BACKGROUND using `pane="tilePane"`)
    folium.Circle(
        location=business_point,
        radius=1000,  # 1km in meters
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.3,
        popup=folium.Popup(f"Total Population in 1km: {total_population}", max_width=300),
        pane="tilePane"  # Ensures circles stay behind squares (not clickable)
    ).add_to(m)

# Save and display the map
m.save("business_population_map.html")
print("Map saved as business_population_map.html")


Map saved as business_population_map.html


In [17]:
import pandas as pd
import folium
import json
from shapely.geometry import Point, Polygon
import geopandas as gpd

# Load business and population data
business_data = pd.read_csv("../../Collected Data/Old data/restaurants_dataset_cleaned2.csv")  
population_data = pd.read_csv("../../Cleaned Data/Cleaned_population_data.csv")  

# Create Folium map centered around business locations
map_center = [business_data['POI Latitude'].mean(), business_data['POI Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Function to convert geometry_region to a valid Polygon
def convert_to_polygon(geometry_string):
    try:
        coordinates = json.loads(geometry_string)  # Convert string to list
        if isinstance(coordinates, list) and len(coordinates) > 0:
            return Polygon([tuple(coord) for coord in coordinates[0]])  # Ensure tuple format
    except Exception as e:
        print(f"Error converting polygon: {e}")
    return None  # Return None if conversion fails

# Convert population polygons 
population_data['geometry_region'] = population_data['geometry_region'].apply(convert_to_polygon)
population_data = population_data.dropna(subset=['geometry_region'])  # Remove invalid polygons

# Plot population squares FIRST 
for _, row in population_data.iterrows():
    polygon = row['geometry_region']
    population_count = row['population_count']
    
    folium.GeoJson(
        {
            "type": "Feature",
            "geometry": {"type": "Polygon", "coordinates": [list(polygon.exterior.coords)]},
            "properties": {"population": population_count}
        },
        name="Population Squares",
        style_function=lambda feature: {
            "fillColor": "#ff7800",
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.5
        },
        pane="overlayPane"  # Moves population squares to foreground 
    ).add_to(m)
    
    # Add label to population squares without clicking
    # Adding a folium.Marker as a label for each population square
    folium.Marker(
        location=[polygon.centroid.y, polygon.centroid.x],  # Center of the square
        icon=folium.DivIcon(html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{population_count}</div>'),
        pane="overlayPane"  # Label appears above the circle
    ).add_to(m)

# Plot businesses and their 1km buffer 
for _, row in business_data.iterrows():
    business_point = [row['POI Latitude'], row['POI Longitude']]
    business_location = Point(row['POI Longitude'], row['POI Latitude'])  # Convert to Shapely Point
    
    # Create a 1km buffer
    buffer_radius = 0.009  # Approx 1km in degrees (for lat/lon)
    business_buffer = business_location.buffer(buffer_radius)
    
    # Calculate total population within 1km buffer
    total_population = sum(
        pop_row['population_count'] for _, pop_row in population_data.iterrows() 
        if pop_row['geometry_region'].intersects(business_buffer)
    )

    # Marker for the business
    folium.Marker(
        location=business_point,
        popup=row['POI Name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)
    
    # 1km radius circle (SEND THIS TO BACKGROUND using `pane="tilePane"`)
    folium.Circle(
        location=business_point,
        radius=1000,  # 1km in meters
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.3,
        popup=folium.Popup(f"Total Population in 1km: {total_population}", max_width=300),
        pane="tilePane"  # Ensures circles stay behind squares (not clickable)
    ).add_to(m)

# Save and display the map
m.save("business_population_map.html")
print("Map saved as business_population_map.html")


Map saved as business_population_map.html
